# Post-processing UQ.2 

In [ ]:
import geopandas as gpd
import matplotlib.pyplot as plt
from pathlib import Path
import folium
import pandas as pd
from branca.colormap import LinearColormap

root_dir = Path("user_question_2")
path_result =  root_dir.joinpath("example_redundancy_multi.gpkg")
path_csv = root_dir.joinpath("event_weights.csv")


gdf = gpd.read_file(path_result, driver="JSON")
gdf = gdf.rename(columns={"diff_length": "TC_0002_diff_length"})
gdf.fillna(0, inplace=True)

gdf.head()

In [ ]:
# import the scenario names and their respective weights:
scenarios_weights_df = pd.read_csv(path_csv)
number_of_scenarios = len(scenarios_weights_df)

columns_to_count = []
for name in scenarios_weights_df['name'].to_list():
    actual = name + "_diff_length"
    print(actual)
    if actual in gdf.columns:
        
        columns_to_count.append(actual)


weights_dict = {}
highest_weight = 0
for _, row in scenarios_weights_df.iterrows():
    weights_dict[row['name']+ "_diff_length"] = row['weight']
    if row['weight'] > highest_weight:
        highest_weight = row['weight']
        most_likely_event = row['name']+ "_diff_length"

print(scenarios_weights_df['name'].to_list())
print(columns_to_count)

print(weights_dict)
print(most_likely_event, highest_weight)

In [ ]:
# Parametrization:
result_type = "max_detour"   #One of "max_detour" , "most_likely_event" , "most_likely_detour" , "expected_detour"

In [ ]:
# Add a new column with the count of non-zero values across specified columns

gdf['total_risk'] = 0
for index, row in gdf.iterrows():
    total_risk = 0
    for col in columns_to_count:
        weight = weights_dict[col]
        risk = weight * row[col]
        total_risk += risk
    gdf.at[index, 'total_risk'] = total_risk



In [ ]:
### RESULT map for total risk 
# Create a Folium pop_diff_map centered around the mean of the geometry
center = [
    gdf['geometry'].centroid.y.mean(),
    gdf['geometry'].centroid.x.mean()
]
pop_diff_map = folium.Map(location=center, zoom_start=5, control_scale=True, tiles="cartodbpositron")
 
 
var = 'total_risk'

vmin=(gdf[var]).min()
vmax=(gdf[var]).max()
# Classify values into ranges
ranges = [vmin, vmin+((vmax-vmin)/4), vmin+2*((vmax-vmin)/4), vmin+3*((vmax-vmin)/4) , vmax]

ranges=sorted(ranges)
# Create a colormap from green to green to red using the overall min and max values
colormap = LinearColormap(colors=['lightgreen', 'orange', 'darkred'],
                          vmin=0,
                          vmax=vmax
                          )
 
 



 
    # Add target_optimal_routes_with_hazard_gdf with created Colormap to feature_group
for idx, row in gdf.iterrows():
    value = row[var]
    # color = 'blue' if (row['count'] > 0)  else 'lightgrey' 

    if row[var] == 0:
        color = 'lightgrey'
    else:
        color = colormap(row[var])

    # Extracting coordinates from MultiLineString
    coordinates = []
    if row['geometry'].geom_type == 'MultiLineString':
        for line in row['geometry']:
            coords = [(coord[1], coord[0]) for coord in line.coords]
            coordinates.extend(coords)
    else:
        coordinates = [(coord[1], coord[0]) for coord in row['geometry'].coords]
    # Create a popup with data
    popup_content = f"total risk: {value} meters"
    popup = folium.Popup(popup_content, max_width=300)

    folium.PolyLine(
        locations=coordinates,
        color=color,
        weight=3,
        opacity=1,
        popup=popup
    ).add_to(pop_diff_map)


    #  from branca.colormap import LinearColormap


# colormap = branca.colormap.linear.YlOrRd_09.scale(0, 8500)
colormap = colormap.to_step(index=ranges)
colormap.caption = 'Total risk (m)'
colormap.add_to(pop_diff_map)

#pop_diff_map.save(root_dir / "map_figure.html")

pop_diff_map

In [ ]:
##
from branca.colormap import LinearColormap
 
# Create a Folium pop_diff_map centered around the mean of the geometry
center = [
    gdf['geometry'].centroid.y.mean(),
    gdf['geometry'].centroid.x.mean()
]
pop_diff_map = folium.Map(location=center, zoom_start=5, control_scale=True, tiles="cartodbpositron")
 
 
var = most_likely_event

vmin=(gdf[var]).min()
vmax=(gdf[var]).max()
# Classify values into ranges
ranges = [vmin, vmin+((vmax-vmin)/4), vmin+2*((vmax-vmin)/4), vmin+3*((vmax-vmin)/4) , vmax]

ranges=sorted(ranges)
# Create a colormap from green to green to red using the overall min and max values
colormap = LinearColormap(colors=['lightgreen', 'orange', 'darkred'],
                          vmin=0,
                          vmax=vmax
                          )
 
 


   

 
    # Add target_optimal_routes_with_hazard_gdf with created Colormap to feature_group
for idx, row in gdf.iterrows():
    value = row[var]
    # color = 'blue' if (row['count'] > 0)  else 'lightgrey' 

    if row[var] == 0:
        color = 'lightgrey'
    else:
        color = colormap(row[var])

    # Extracting coordinates from MultiLineString
    coordinates = []
    if row['geometry'].geom_type == 'MultiLineString':
        for line in row['geometry']:
            coords = [(coord[1], coord[0]) for coord in line.coords]
            coordinates.extend(coords)
    else:
        coordinates = [(coord[1], coord[0]) for coord in row['geometry'].coords]
    # Create a popup with data
    popup_content = f"Diff length = {value} m (event {var})"
    popup = folium.Popup(popup_content, max_width=300)

    folium.PolyLine(
        locations=coordinates,
        color=color,
        weight=3,
        opacity=1,
        popup=popup
    ).add_to(pop_diff_map)


    #  from branca.colormap import LinearColormap


# colormap = branca.colormap.linear.YlOrRd_09.scale(0, 8500)
colormap = colormap.to_step(index=ranges)
colormap.caption = 'Diff length (m)'
colormap.add_to(pop_diff_map)

#pop_diff_map.save(root_dir / "map_figure.html")

pop_diff_map